In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aksharantar-sampled/aksharantar_sampled/brx/brx_test.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/brx/brx_valid.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/brx/brx_train.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/tam/tam_valid.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/tam/tam_train.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/tam/tam_test.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/mni/mni_valid.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/mni/mni_test.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/mni/mni_train.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/urd/urd_train.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/urd/urd_valid.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/urd/urd_test.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/kok/kok_valid.csv
/kaggle/input/aksharantar-sampled/aksharantar_sampled/kok/kok_train.

In [2]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import gc
import random

In [3]:
if torch.cuda.is_available():
    # If CUDA is available, use a CUDA device
    device = torch.device("cuda")
else:
    # If CUDA is not available, use the CPU
    device = torch.device("cpu")

In [4]:
device

device(type='cuda')

# Preprocessing

In [5]:
traindata = pd.read_csv('/kaggle/input/aksharantar-sampled/aksharantar_sampled/hin/hin_train.csv',names= ['English','Hindi'],header = None)

In [6]:
testdata = pd.read_csv('/kaggle/input/aksharantar-sampled/aksharantar_sampled/hin/hin_test.csv',names = ['English','Hindi'],header = None)

In [7]:
valdata = pd.read_csv('/kaggle/input/aksharantar-sampled/aksharantar_sampled/hin/hin_valid.csv',names = ['English','Hindi'],header = None)

In [8]:
traindata

,English,Hindi
0,shastragaar,शस्त्रागार
1,bindhya,बिन्द्या
2,kirankant,किरणकांत
3,yagyopaveet,यज्ञोपवीत
4,ratania,रटानिया
...,...,...
51195,toned,टोंड
51196,mutanaazaa,मुतनाज़ा
51197,asahmaton,असहमतों
51198,sulgaayin,सुलगायीं


In [9]:
def tokenize(word):
    tokens = []
    for x in word:
        tokens.append(x)
    return tokens

In [10]:
tokenize(traindata.iloc[0]['Hindi'])

['श', 'स', '्', 'त', '्', 'र', 'ा', 'ग', 'ा', 'र']

In [11]:
max_eng_len = 0
max_hin_len = 0

In [12]:
test_max_eng_len = 0
test_max_hin_len = 0

In [13]:
for x in range(len(testdata)):
    temp = 0
    for y in testdata.iloc[x]['English']:
        temp+=1
    test_max_eng_len = max(test_max_eng_len,temp)
print(test_max_eng_len)

26


In [14]:
for x in range(len(testdata)):
    temp = 0
    for y in testdata.iloc[x]['Hindi']:
        temp +=1
    test_max_hin_len = max(test_max_hin_len,temp)
print(test_max_hin_len)

20


In [15]:
val_max_eng_len = 0
val_max_hin_len = 0

In [16]:
for x in range(len(valdata)):
    temp = 0
    for y in valdata.iloc[x]['English']:
        temp+=1
    val_max_eng_len = max(val_max_eng_len,temp)
print(val_max_eng_len)

22


In [17]:
for x in range(len(valdata)):
    temp = 0
    for y in valdata.iloc[x]['Hindi']:
        temp+=1
    val_max_hin_len = max(val_max_hin_len,temp)
print(val_max_hin_len)

20


In [18]:
English_vocab = []
for x in range(len(traindata)):
    temp = 0
    for y in traindata.iloc[x]['English']:
        temp += 1
        if y not in English_vocab:
            English_vocab.append(y)
    if(temp>max_eng_len):
        max_eng_len = max(max_eng_len,temp)
print(sorted(English_vocab))
print(max_eng_len)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
24


In [19]:
Hindi_vocab = []
for x in range(len(traindata)):
    temp = 0
    for y in traindata.iloc[x]['Hindi']:
        temp += 1
        if y not in Hindi_vocab:
            Hindi_vocab.append(y)
    max_hin_len = max(temp,max_hin_len)
for x in range(len(testdata)):
    for y in testdata.iloc[x]['Hindi']:
        if y not in Hindi_vocab:
            print(y)
            Hindi_vocab.append(y)

ॊ


In [20]:
print(max_hin_len)

20


In [21]:
English_vocab = sorted(English_vocab)
Hindi_vocab = sorted(Hindi_vocab)

In [22]:
Eng_dict = {}
reverse_Eng = {}

for x in range(len(English_vocab)):
    Eng_dict[English_vocab[x]] = x+3
    reverse_Eng[x+3] = English_vocab[x]
Eng_dict['<sow>'] = 0
Eng_dict['<eow>'] = 1
Eng_dict['<pad>'] = 2
reverse_Eng[0] = '<sow>'
reverse_Eng[1] = '<eow>'
reverse_Eng[2] = '<pad>'

print(Eng_dict)
print(reverse_Eng)

{'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, '<sow>': 0, '<eow>': 1, '<pad>': 2}
{3: 'a', 4: 'b', 5: 'c', 6: 'd', 7: 'e', 8: 'f', 9: 'g', 10: 'h', 11: 'i', 12: 'j', 13: 'k', 14: 'l', 15: 'm', 16: 'n', 17: 'o', 18: 'p', 19: 'q', 20: 'r', 21: 's', 22: 't', 23: 'u', 24: 'v', 25: 'w', 26: 'x', 27: 'y', 28: 'z', 0: '<sow>', 1: '<eow>', 2: '<pad>'}


In [23]:
Hin_dict = {}
reverse_Hin = {}
for x in range(len(Hindi_vocab)):
    Hin_dict[Hindi_vocab[x]] = x+3
    reverse_Hin[x+3] = Hindi_vocab[x]
Hin_dict['<sow>'] = 0
Hin_dict['<eow>'] = 1
Hin_dict['<pad>'] = 2
reverse_Hin[0] = '<sow>'
reverse_Hin[1] = '<eow>'
reverse_Hin[2] = '<pad>'
print(Hin_dict)
print(reverse_Hin)

{'ँ': 3, 'ं': 4, 'ः': 5, 'अ': 6, 'आ': 7, 'इ': 8, 'ई': 9, 'उ': 10, 'ऊ': 11, 'ऋ': 12, 'ए': 13, 'ऐ': 14, 'ऑ': 15, 'ओ': 16, 'औ': 17, 'क': 18, 'ख': 19, 'ग': 20, 'घ': 21, 'ङ': 22, 'च': 23, 'छ': 24, 'ज': 25, 'झ': 26, 'ञ': 27, 'ट': 28, 'ठ': 29, 'ड': 30, 'ढ': 31, 'ण': 32, 'त': 33, 'थ': 34, 'द': 35, 'ध': 36, 'न': 37, 'प': 38, 'फ': 39, 'ब': 40, 'भ': 41, 'म': 42, 'य': 43, 'र': 44, 'ल': 45, 'ळ': 46, 'व': 47, 'श': 48, 'ष': 49, 'स': 50, 'ह': 51, '़': 52, 'ऽ': 53, 'ा': 54, 'ि': 55, 'ी': 56, 'ु': 57, 'ू': 58, 'ृ': 59, 'ॅ': 60, 'े': 61, 'ै': 62, 'ॉ': 63, 'ॊ': 64, 'ो': 65, 'ौ': 66, '्': 67, '<sow>': 0, '<eow>': 1, '<pad>': 2}
{3: 'ँ', 4: 'ं', 5: 'ः', 6: 'अ', 7: 'आ', 8: 'इ', 9: 'ई', 10: 'उ', 11: 'ऊ', 12: 'ऋ', 13: 'ए', 14: 'ऐ', 15: 'ऑ', 16: 'ओ', 17: 'औ', 18: 'क', 19: 'ख', 20: 'ग', 21: 'घ', 22: 'ङ', 23: 'च', 24: 'छ', 25: 'ज', 26: 'झ', 27: 'ञ', 28: 'ट', 29: 'ठ', 30: 'ड', 31: 'ढ', 32: 'ण', 33: 'त', 34: 'थ', 35: 'द', 36: 'ध', 37: 'न', 38: 'प', 39: 'फ', 40: 'ब', 41: 'भ', 42: 'म', 43: 'य', 44: 'र', 45: 'ल', 46: 

In [24]:
def Eng_tokenize(word):
    tokens = []
    for x in word:
        tokens.append(Eng_dict[x])
    for x in range(len(tokens),max_eng_len):
        tokens.append(Eng_dict['<pad>'])
    return tokens

In [25]:
Eng_tokenize(traindata.iloc[0]['English'])

[21, 10, 3, 21, 22, 20, 3, 9, 3, 3, 20, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [26]:
def Hin_tokenize(word):
    tokens = []
    for x in word:
        tokens.append(Hin_dict[x])
    tokens.append(Hin_dict['<eow>'])
    for x in range(len(tokens),max_hin_len+1):
        tokens.append(Hin_dict['<pad>'])
    return tokens

In [27]:
eng_word = []
hin_word = []
for x in range(len(traindata)):
    eng_word.append(Eng_tokenize(traindata.iloc[x]['English']))
    hin_word.append(Hin_tokenize(traindata.iloc[x]['Hindi']))

In [28]:
eng_word = torch.tensor(eng_word)
hin_word = torch.tensor(hin_word)

In [29]:
max_hin_len

20

In [30]:
max_hin_len += 1
test_max_hin_len += 1
val_max_hin_len += 1

In [31]:
max_hin_len

21

In [32]:
def test_Eng_tokenize(word):
    tokens = []
    for x in word:
        tokens.append(Eng_dict[x])
    for x in range(len(tokens),test_max_eng_len):
        tokens.append(Eng_dict['<pad>'])
    return tokens
def test_Hin_tokenize(word):
    tokens = []
    for x in word:
        tokens.append(Hin_dict[x])
    tokens.append(Hin_dict['<eow>'])
    for x in range(len(tokens),test_max_hin_len):
        tokens.append(Hin_dict['<pad>'])
    return tokens
def val_Eng_tokenize(word):
    tokens = []
    for x in word:
        tokens.append(Eng_dict[x])
    for x in range(len(tokens),val_max_eng_len):
        tokens.append(Eng_dict['<pad>'])
    return tokens
def val_Hin_tokenize(word):
    tokens = []
    for x in word:
        tokens.append(Hin_dict[x])
    tokens.append(Hin_dict['<eow>'])
    for x in range(len(tokens),val_max_hin_len):
        tokens.append(Hin_dict['<pad>'])
    return tokens
val_eng_word = []
val_hin_word = []
for x in range(len(valdata)):
    val_eng_word.append(val_Eng_tokenize(valdata.iloc[x]['English']))
    val_hin_word.append(val_Hin_tokenize(valdata.iloc[x]['Hindi']))
val_eng_word = torch.tensor(val_eng_word)
val_hin_word = torch.tensor(val_hin_word)
test_eng_word = []
test_hin_word = []
for x in range(len(testdata)):
    test_eng_word.append(test_Eng_tokenize(testdata.iloc[x]['English']))
    test_hin_word.append(test_Hin_tokenize(testdata.iloc[x]['Hindi']))
test_eng_word = torch.tensor(test_eng_word)
test_hin_word = torch.tensor(test_hin_word)

# Encoder, Decoder and Attention

In [33]:
class Encoder(nn.Module):
    def __init__(self,char_embed_size,hidden_size,no_of_layers,dropout,rnn):
        super(Encoder,self).__init__()
        self.layer = no_of_layers
        self.rnn = rnn
        self.embedding = nn.Embedding(len(Eng_dict),char_embed_size).to(device)
        self.embedding.weight.requires_grad = True
        self.drop = nn.Dropout(dropout)
        self.LSTM = nn.LSTM(char_embed_size,hidden_size,self.layer,batch_first = True,bidirectional = True).to(device)
        self.RNN = nn.RNN(char_embed_size,hidden_size,self.layer,batch_first = True,bidirectional = True).to(device)
        self.GRU = nn.GRU(char_embed_size,hidden_size,self.layer,batch_first = True,bidirectional = True).to(device)
    def forward(self,input,hidden,cell):
        embedded = self.embedding(input)
        embedded1 = self.drop(embedded)
        cell1 = cell
        if(self.rnn == 'RNN'):
            output,hidden1 = self.RNN(embedded1,hidden)
        elif(self.rnn == 'LSTM'):
            output,(hidden1,cell1) = self.LSTM(embedded1,(hidden,cell))
        elif(self.rnn == 'GRU'):
            output,hidden1 = self.GRU(embedded1,hidden)
        return output,(hidden1,cell1)

In [34]:
class Decoder(nn.Module):
    def __init__(self,char_embed_size,hidden_size,no_of_layers,dropout,batchsize,rnn):
        super(Decoder,self).__init__()
        self.layer = no_of_layers
        self.batchsize = batchsize
        self.hidden_size = hidden_size
        self.rnn = rnn
        self.embedding = nn.Embedding(len(Hin_dict),char_embed_size).to(device)
        self.drop = nn.Dropout(dropout)
        self.embedding.weight.requires_grad = True
        self.LSTM = nn.LSTM(char_embed_size + hidden_size*2,hidden_size,self.layer,batch_first = True).to(device)
        self.RNN = nn.RNN(char_embed_size + hidden_size*2,hidden_size,self.layer,batch_first = True).to(device)
        self.GRU = nn.GRU(char_embed_size + hidden_size*2,hidden_size,self.layer,batch_first = True).to(device)
        #2*hidden_size
        self.linear = nn.Linear(hidden_size,len(Hin_dict),bias=True).to(device)
        # dim = 2 
        self.softmax = nn.Softmax(dim = 2).to(device)
    def forward(self,input,hidden,cell,OGhidden):
        embedded = self.embedding(input)
        embedded1 = torch.cat((embedded,OGhidden[0].resize(self.batchsize,1,self.hidden_size),OGhidden[1].resize(self.batchsize,1,self.hidden_size)),dim = 2)
        embedded2 = self.drop(embedded1)
        cell1 = cell
        if(self.rnn == 'LSTM'):
            output,(hidden1,cell1) = self.LSTM(embedded2,(hidden,cell))
        elif(self.rnn == 'RNN'):
            output,hidden1 = self.RNN(embedded2,hidden)
        elif(self.rnn == 'GRU'):
            output,hidden1 = self.GRU(embedded2,hidden)
        output1 = self.linear(output)
        return output1,(hidden1,cell1)
        output2 = self.softmax(output1)
        return output2,hidden11
        
    #changed GRU char_embed_size
    #changed forward embedded

In [35]:
class Attention(nn.Module):
    def __init__(self,char_embed_size,hidden_size,no_of_layers,dropout,batchsize,rnn):
        super(Attention,self).__init__()
        self.layer = no_of_layers
        self.batchsize = batchsize
        self.hidden_size = hidden_size
        self.rnn = rnn
        self.embedding = nn.Embedding(len(Hin_dict),char_embed_size).to(device)
        self.drop = nn.Dropout(dropout)
        self.embedding.weight.requires_grad = True
        self.U = nn.Linear(hidden_size,hidden_size,bias = False).to(device)
        self.W = nn.Linear(hidden_size,hidden_size,bias = False).to(device)
        self.V = nn.Linear(hidden_size,1,bias = False).to(device)
        self.LSTM = nn.LSTM(char_embed_size + hidden_size,hidden_size,self.layer,batch_first = True).to(device)
        self.RNN = nn.RNN(char_embed_size + hidden_size,hidden_size,self.layer,batch_first = True).to(device)
        self.GRU = nn.GRU(char_embed_size + hidden_size,hidden_size,self.layer,batch_first = True).to(device) 
        self.linear = nn.Linear(hidden_size,len(Hin_dict),bias=True).to(device)
        self.softmax = nn.Softmax(dim = 2).to(device)
    def forward(self,input,hidden,cell,encoder_outputs):
        embedded = self.embedding(input)
        temp1 = self.U(encoder_outputs)
        temp2 = self.W(hidden[-1])
        add = temp1 + temp2.resize(self.batchsize,1,self.hidden_size)
        tanh = F.tanh(add)
        ejt = self.V(tanh)
        ajt = nn.Softmax(dim = 1)(ejt)
        ct = torch.zeros(self.batchsize,1,self.hidden_size).to(device)
        ct = torch.bmm(ajt.transpose(1,2),encoder_outputs)
        final_input = torch.cat((embedded,ct),dim = 2)
        cell1 = cell
        if(self.rnn == 'LSTM'):
            output,(hidden1,cell1) = self.LSTM(final_input,(hidden,cell))
        elif(self.rnn == 'RNN'):
            output,hidden1 = self.RNN(final_input,hidden)
        elif(self.rnn == 'GRU'):
            output,hidden1 = self.GRU(final_input,hidden)
        output1 = self.linear(output)
        return output1,(hidden1,cell1)

# Attentiontrain, train, Evaluate, valevaluate

In [36]:
def Evaluate(attention,test_eng_word,test_hin_word,encoder,decoder,batchsize,hidden_size,char_embed_size,no_of_layers):
    with torch.no_grad():
        total_loss = 0
        total_acc = 0
        en_hidden = torch.zeros(2*no_of_layers,batchsize,hidden_size).to(device)
        en_cell = torch.zeros(2*no_of_layers,batchsize,hidden_size).to(device)
        for x in range(0,len(testdata),batchsize):
            loss = 0
            input_tensor = test_eng_word[x:x+batchsize].to(device)
#             en_hidden = torch.zeros(2*no_of_layers,batchsize,hidden_size).to(device)
            if(input_tensor.size()[0] < batchsize):
                break
            output,(hidden,cell) = encoder.forward(input_tensor,en_hidden,en_cell)
            del(input_tensor)
            output = torch.split(output,[hidden_size,hidden_size],dim = 2)
            output = torch.add(output[0],output[1])/2
            input2 = []
            for y in range(batchsize):
                input2.append([0])
            input2 = torch.tensor(input2).to(device)
            hidden = hidden.resize(2,no_of_layers,batchsize,hidden_size)
            hidden1 = torch.add(hidden[0],hidden[1])/2
#             hidden1 = hidden[0]
            cell = cell.resize(2,no_of_layers,batchsize,hidden_size)
            cell1 = torch.add(cell[0],cell[1])/2
#             cell1 = cell[0]
            OGhidden = hidden1
            predicted = []
            predictions = []
            if(attention == True):
                temp = output
            else:
                temp = OGhidden
            for i in range(test_max_hin_len):
                output1,(hidden1,cell1) = decoder.forward(input2,hidden1,cell1,temp)
                predicted.append(output1)
                output2 = decoder.softmax(output1)
                output3 = torch.argmax(output2,dim = 2)
                predictions.append(output3)
                input2 = output3
            predicted = torch.cat(tuple(x for x in predicted),dim =1).to(device).resize(test_max_hin_len*batchsize,len(Hin_dict))
            predictions = torch.cat(tuple(x for x in predictions),dim =1).to(device)
            total_acc += accuracy(test_hin_word[x:x+batchsize].to(device),predictions,x)
            loss  = nn.CrossEntropyLoss(reduction = 'sum')(predicted,test_hin_word[x:x+batchsize].reshape(-1).to(device))
            with torch.no_grad():
                total_loss += loss.item()
#         print('test_loss')
        test_loss = total_loss/(len(testdata)*test_max_hin_len)
#         print(test_loss)
#         print('test_accuracy')
        test_accuracy = (total_acc/len(testdata))*100
#         print(test_accuracy)
        del(predictions)
        del(predicted)
        del(input2)
        del(output1)
        del(output2)
        del(output3)
        del(hidden1)
        del(cell1)
        del(OGhidden)
        del(output)
        del(cell)
        return test_loss,test_accuracy

In [56]:
import math

def tempvalevaluate(attention, eng_word, hin_word, encoder, decoder, batch_size, hidden_size, char_embed_size, no_of_layers, beam_width):
    with torch.no_grad():
        total_loss = 0
        total_acc = 0
        for x in range(0, len(eng_word), batch_size):
            input_tensor = eng_word[x:x+batch_size].to(device)
            if input_tensor.size()[0] < batch_size:
                break
            en_hidden = torch.zeros(2*no_of_layers, batch_size, hidden_size).to(device)
            en_cell = torch.zeros(2*no_of_layers, batch_size, hidden_size).to(device)
            output, (hidden, cell) = encoder.forward(input_tensor, en_hidden, en_cell)
            del(input_tensor)
            del(en_hidden)
            del(en_cell)
            output = torch.split(output, [hidden_size, hidden_size], dim=2)
            output = torch.add(output[0], output[1])/2
            hidden = hidden.resize(2, no_of_layers, batch_size, hidden_size)
            hidden1 = torch.add(hidden[0], hidden[1])/2
            cell = cell.resize(2, no_of_layers, batch_size, hidden_size)
            cell1 = torch.add(cell[0], cell[1])/2
            OGhidden = hidden1
            if attention:
                temp = output
            else:
                temp = OGhidden
            # initialize beam
            beam = [(0, [0]*batchsize, hidden1, cell1)]  # Start with <SOS> token
            for i in range(val_max_hin_len):
                new_beam = []
                for score, sequence, hidden1, cell1 in beam:
                    if sequence[-1] == 1:  # <EOS> token
                        new_beam.append((score, sequence, hidden1, cell1))
                        continue
                    input2 = torch.tensor([sequence[-1]]).unsqueeze(0).to(device)
                    output1, (hidden1, cell1) = decoder.forward(input2, hidden1, cell1, temp)
                    output2 = decoder.softmax(output1)
                    top_probs, top_indices = output2.topk(beam_width)
                    for j in range(beam_width):
                        new_seq = sequence + [top_indices[0][j].item()]
                        new_score = score - math.log(top_probs[0][j].item())
                        new_beam.append((new_score, new_seq, hidden1, cell1))
                new_beam.sort()
                beam = new_beam[:beam_width]
            predicted = torch.tensor([seq for score, seq, hidden1, cell1 in beam]).to(device)
            predicted = predicted[:, 1:]  # remove <SOS> token
            predicted = predicted.reshape(-1, predicted.size(2))
            predictions = predicted.argmax(dim=1)
            total_acc += accuracy(hin_word[x:x+batch_size].to(device), predictions, x)
            loss = nn.CrossEntropyLoss(reduction='sum')(predicted, hin_word[x:x+batch_size].reshape(-1).to(device))
            total_loss += loss.item()
            del(predictions)
            del(predicted)
            del(hidden1)
            del(cell1)
        validation_loss = total_loss/(len(valdata)*val_max_hin_len)
        validation_accuracy = (total_acc/len(valdata))*100


In [57]:
batchsize,hidden_size,char_embed_size,no_of_layers,epochs,rnn = 512,256,256,8,10,'LSTM'
train(batchsize,hidden_size,char_embed_size,no_of_layers,0.4,epochs,rnn)

<bound method Module.parameters of Encoder(
  (embedding): Embedding(29, 256)
  (drop): Dropout(p=0.4, inplace=False)
  (LSTM): LSTM(256, 256, num_layers=8, batch_first=True, bidirectional=True)
  (RNN): RNN(256, 256, num_layers=8, batch_first=True, bidirectional=True)
  (GRU): GRU(256, 256, num_layers=8, batch_first=True, bidirectional=True)
)>
<bound method Module.parameters of Decoder(
  (embedding): Embedding(68, 256)
  (drop): Dropout(p=0.4, inplace=False)
  (LSTM): LSTM(768, 256, num_layers=8, batch_first=True)
  (RNN): RNN(768, 256, num_layers=8, batch_first=True)
  (GRU): GRU(768, 256, num_layers=8, batch_first=True)
  (linear): Linear(in_features=256, out_features=68, bias=True)
  (softmax): Softmax(dim=2)
)>
0


RuntimeError: Sizes of tensors must match except in dimension 2. Expected size 1 but got size 512 for tensor number 1 in the list.

In [37]:
def valevaluate(attention,val_eng_word,val_hin_word,encoder,decoder,batchsize,hidden_size,char_embed_size,no_of_layers):
    with torch.no_grad():
        total_loss = 0
        total_acc = 0
        for x in range(0,len(valdata),batchsize):
            loss = 0
            input_tensor = val_eng_word[x:x+batchsize].to(device)
#             en_hidden = torch.zeros(2*no_of_layers,batchsize,hidden_size).to(device)
            if(input_tensor.size()[0] < batchsize):
                break
            en_hidden = torch.zeros(2*no_of_layers,batchsize,hidden_size).to(device)
            en_cell = torch.zeros(2*no_of_layers,batchsize,hidden_size).to(device)
            output,(hidden,cell) = encoder.forward(input_tensor,en_hidden,en_cell)
            del(input_tensor)
            del(en_hidden)
            del(en_cell)
            output = torch.split(output,[hidden_size,hidden_size],dim = 2)
            output = torch.add(output[0],output[1])/2
            input2 = []
            for y in range(batchsize):
                input2.append([0])
            input2 = torch.tensor(input2).to(device)
            hidden = hidden.resize(2,no_of_layers,batchsize,hidden_size)
            hidden1 = torch.add(hidden[0],hidden[1])/2
#             hidden1 = hidden[0]
            cell = cell.resize(2,no_of_layers,batchsize,hidden_size)
            cell1 = torch.add(cell[0],cell[1])/2
#             cell1 = cell[0]
            OGhidden = hidden1
            predicted = []
            predictions = []
            if(attention == True):
                temp = output
            else:
                temp = OGhidden
            for i in range(val_max_hin_len):
                output1,(hidden1,cell1) = decoder.forward(input2,hidden1,cell1,temp)
                predicted.append(output1)
                output2 = decoder.softmax(output1)
                output3 = torch.argmax(output2,dim = 2)
                predictions.append(output3)
                input2 = output3
            predicted = torch.cat(tuple(x for x in predicted),dim =1).to(device).resize(val_max_hin_len*batchsize,len(Hin_dict))
            predictions = torch.cat(tuple(x for x in predictions),dim =1).to(device)
            total_acc += accuracy(val_hin_word[x:x+batchsize].to(device),predictions,x)
            loss  = nn.CrossEntropyLoss(reduction = 'sum')(predicted,val_hin_word[x:x+batchsize].reshape(-1).to(device))
            with torch.no_grad():
                total_loss += loss.item()
        validation_loss = total_loss/(len(valdata)*val_max_hin_len)
        validation_accuracy = (total_acc/len(valdata))*100
        del(predictions)
        del(predicted)
        del(input2)
        del(output1)
        del(output2)
        del(output3)
        del(hidden1)
        del(cell1)
        del(OGhidden)
        del(output)
        del(cell)
        return validation_loss,validation_accuracy

In [49]:
def attentiontrain(batchsize,hidden_size,char_embed_size,no_of_layers,dropout,epochs,rnn):
    gc.collect()
    torch.autograd.set_detect_anomaly(True)
    encoder = Encoder(char_embed_size,hidden_size,no_of_layers,dropout,rnn).to(device)
    decoder = Attention(char_embed_size,hidden_size,no_of_layers,dropout,batchsize,rnn).to(device)
    print(encoder.parameters)
    print(decoder.parameters)
    opt_encoder = optim.Adam(encoder.parameters(),lr = 0.001)
    opt_decoder  = optim.Adam(decoder.parameters(),lr = 0.001)
    teacher_ratio = 0.5
    for _ in range(epochs):
        torch.cuda.empty_cache()
        print(_)
        total_loss = 0
        total_acc = 0
        for x in range(0,len(traindata),batchsize):
            loss = 0
            opt_encoder.zero_grad()
            opt_decoder.zero_grad()
            input_tensor = eng_word[x:x+batchsize].to(device)
            en_hidden = torch.zeros(2*no_of_layers,batchsize,hidden_size).to(device)
            en_cell = torch.zeros(2*no_of_layers,batchsize,hidden_size).to(device)
            if(input_tensor.size()[0] < batchsize):
                break
            output,(hidden,cell) = encoder.forward(input_tensor,en_hidden,en_cell)
            output = torch.split(output,[hidden_size,hidden_size],dim = 2)
            output = torch.add(output[0],output[1])/2
            input2 = []
            for y in range(batchsize):
                input2.append([0])
            input2 = torch.tensor(input2).to(device)
            hidden = hidden.resize(2,no_of_layers,batchsize,hidden_size)
            hidden1 = torch.add(hidden[0],hidden[1])/2
            cell = cell.resize(2,no_of_layers,batchsize,hidden_size)
            cell1 = torch.add(cell[0],cell[1])/2
            predicted = []
            predictions = []
#             use_teacher_forcing = True if random.random() < teacher_ratio else False
            for i in range(max_hin_len):
                use_teacher_forcing = True if random.random() < teacher_ratio else False
                output1,(hidden1,cell1) = decoder.forward(input2,hidden1,cell1,output)
                predicted.append(output1)
                output2 = decoder.softmax(output1)
                output3 = torch.argmax(output2,dim = 2)
                predictions.append(output3)
                if(use_teacher_forcing):
                    input2 = hin_word[x:x+batchsize,i].to(device).resize(batchsize,1)
                else:
                    input2 = hin_word[x:x+batchsize,i].to(device).resize(batchsize,1)

#             if use_teacher_forcing:
#                 for i in range(max_hin_len):
#                     output1,(hidden1,cell1) = decoder.forward(input2,hidden1,cell1,output)
#                     predicted.append(output1)
#                     output2 = decoder.softmax(output1)
#                     output3 = torch.argmax(output2,dim = 2)
#                     predictions.append(output3)
#                     input2 = hin_word[x:x+batchsize,i].to(device).resize(batchsize,1)
#             else:
#                 for i in range(max_hin_len):
#                     output1,(hidden1,cell1) = decoder.forward(input2,hidden1,cell1,output)
#                     predicted.append(output1)
#                     output2 = decoder.softmax(output1)
#                     output3 = torch.argmax(output2,dim = 2)
#                     predictions.append(output3)
#                     input2 = output3
            
            predicted = torch.cat(tuple(x for x in predicted),dim =1).to(device).resize(max_hin_len*batchsize,len(Hin_dict))
            predictions = torch.cat(tuple(x for x in predictions),dim =1).to(device)
            total_acc += accuracy(hin_word[x:x+batchsize].to(device),predictions,x)
            loss  = nn.CrossEntropyLoss(reduction = 'sum')(predicted,hin_word[x:x+batchsize].reshape(-1).to(device))
            with torch.no_grad():
                total_loss += loss.item()
            loss.backward(retain_graph = True)
            torch.nn.utils.clip_grad_norm_(encoder.parameters(),max_norm = 1)
            torch.nn.utils.clip_grad_norm_(decoder.parameters(),max_norm = 1)
            opt_encoder.step()
            opt_decoder.step()
        del(input_tensor)
        del(en_hidden)
        del(en_cell)
        del(predictions)
        del(predicted)
        del(input2)
        del(output1)
        del(output2)
        del(output3)
        del(hidden)
        del(hidden1)
        del(cell1)
        del(output)
        del(cell)
        training_loss = total_loss/(51200*max_hin_len)
        training_accuracy = total_acc/512
        test_loss,test_accuracy = Evaluate(True,test_eng_word,test_hin_word,encoder,decoder,batchsize,hidden_size,char_embed_size,no_of_layers)
        validation_loss,validation_accuracy = valevaluate(True,val_eng_word,val_hin_word,encoder,decoder,batchsize,hidden_size,char_embed_size,no_of_layers)
        wandb.log({'training_accuracy' : training_accuracy, 'validation_accuracy' : validation_accuracy,'training_loss' : training_loss, 'validation_loss' : validation_loss,'epoch':_+1})
#         if(_ >= epochs/2):
#             teacher_ratio = 0
#         teacher_ratio /= 2
    return encoder,decoder

In [39]:
import gc
gc.collect()

23

In [40]:
# batchsize,hidden_size,char_embed_size,no_of_layers,dropout,epochs,rnn
# train(256,512,256,2,0.3,10,'LSTM')

In [40]:
def train(batchsize,hidden_size,char_embed_size,no_of_layers,dropout,epochs,rnn):
    gc.collect()
    torch.autograd.set_detect_anomaly(True)
    encoder = Encoder(char_embed_size,hidden_size,no_of_layers,dropout,rnn).to(device)
    decoder = Decoder(char_embed_size,hidden_size,no_of_layers,dropout,batchsize,rnn).to(device)
    print(encoder.parameters)
    print(decoder.parameters)
    opt_encoder = optim.Adam(encoder.parameters(),lr = 0.001)
    opt_decoder  = optim.Adam(decoder.parameters(),lr = 0.001)
    teacher_ratio = 0.5
#     en_hidden = torch.randn(2*no_of_layers,batchsize,hidden_size).to(device)
    for _ in range(epochs):
        print(_)
        total_loss = 0
        total_acc = 0
        for x in range(0,len(traindata),batchsize):
            loss = 0
            opt_encoder.zero_grad()
            opt_decoder.zero_grad()
            input_tensor = eng_word[x:x+batchsize].to(device)
            en_hidden = torch.zeros(2*no_of_layers,batchsize,hidden_size).to(device)
            en_cell = torch.zeros(2*no_of_layers,batchsize,hidden_size).to(device)
            if(input_tensor.size()[0] < batchsize):
                break
            output,(hidden,cell) = encoder.forward(input_tensor,en_hidden,en_cell)
            del(en_hidden)
            del(en_cell)
            del(input_tensor)
            input2 = []
            for y in range(batchsize):
                input2.append([0])
            input2 = torch.tensor(input2).to(device)
            hidden = hidden.resize(2,no_of_layers,batchsize,hidden_size)
            cell = cell.resize(2,no_of_layers,batchsize,hidden_size)
            hidden1 = torch.add(hidden[0],hidden[1])/2
            cell1 = torch.add(cell[0],cell[1])/2
            OGhidden = hidden1
            predicted = []
            predictions = []
            use_teacher_forcing = True if random.random() < teacher_ratio else False
            if use_teacher_forcing:
                for i in range(max_hin_len):
                    output1,(hidden1,cell1) = decoder.forward(input2,hidden1,cell1,OGhidden)
                    predicted.append(output1)
                    output2 = decoder.softmax(output1)
                    output3 = torch.argmax(output2,dim = 2)
                    predictions.append(output3)
                    input2 = hin_word[x:x+batchsize,i].to(device).resize(batchsize,1)
            else:
                for i in range(max_hin_len):
                    output1,(hidden1,cell1) = decoder.forward(input2,hidden1,cell1,OGhidden)
                    predicted.append(output1)
                    output2 = decoder.softmax(output1)
                    output3 = torch.argmax(output2,dim = 2)
                    predictions.append(output3)
                    input2 = output3
            predicted = torch.cat(tuple(x for x in predicted),dim =1).to(device).resize(max_hin_len*batchsize,len(Hin_dict))
            predictions = torch.cat(tuple(x for x in predictions),dim =1).to(device)
            total_acc += accuracy(hin_word[x:x+batchsize].to(device),predictions,x)
            loss  = nn.CrossEntropyLoss(reduction = 'sum')(predicted,hin_word[x:x+batchsize].reshape(-1).to(device))
            with torch.no_grad():
                total_loss += loss.item()
            loss.backward(retain_graph = True)
            torch.nn.utils.clip_grad_norm_(encoder.parameters(),max_norm = 1)
            torch.nn.utils.clip_grad_norm_(decoder.parameters(),max_norm = 1)
            opt_encoder.step()
            opt_decoder.step()
        del(predictions)
        del(predicted)
        del(input2)
        del(output1)
        del(output2)
        del(output3)
        del(hidden1)
        del(cell1)
        del(OGhidden)
        del(output)
        del(cell)
        training_loss = total_loss/(51200*max_hin_len)
        training_accuracy = total_acc/512
        test_loss,test_accuracy = Evaluate(False,test_eng_word,test_hin_word,encoder,decoder,batchsize,hidden_size,char_embed_size,no_of_layers)
        validation_loss,validation_accuracy = tempvalevaluate(False,val_eng_word,val_hin_word,encoder,decoder,batchsize,hidden_size,char_embed_size,no_of_layers,2)
#         wandb.log({'training_accuracy' : training_accuracy, 'validation_accuracy' : validation_accuracy,'training_loss' : training_loss, 'validation_loss' : validation_loss,'epoch':_+1})
#         if(_ >= epochs/2):
#             teacher_ratio = 0
    return encoder,decoder

In [41]:
def getword(characters):
    return "".join(characters)

In [42]:
def accuracy(target,predictions,flag):
    total = 0
#     if(flag == 0):
#         for x in range(1):
#             print(getword([reverse_Hin[x.item()] for x in target[x]]))
#             print(getword([reverse_Hin[x.item()] for x in predictions[x]]))
    for x in range(len(target)):
        if(torch.equal(target[x],predictions[x])):
            total += 1
    return total

In [43]:
import wandb

In [44]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [67]:
# batchsize,hidden_size,char_embed_size,no_of_layers,epochs,rnn = 1024,256,256,8,10,'LSTM'
# train(batchsize,hidden_size,char_embed_size,no_of_layers,0.4,epochs,rnn)

<bound method Module.parameters of Encoder(
  (embedding): Embedding(29, 256)
  (drop): Dropout(p=0.4, inplace=False)
  (LSTM): LSTM(256, 256, num_layers=8, batch_first=True, bidirectional=True)
  (RNN): RNN(256, 256, num_layers=8, batch_first=True, bidirectional=True)
  (GRU): GRU(256, 256, num_layers=8, batch_first=True, bidirectional=True)
)>
<bound method Module.parameters of Decoder(
  (embedding): Embedding(68, 256)
  (drop): Dropout(p=0.4, inplace=False)
  (LSTM): LSTM(768, 256, num_layers=8, batch_first=True)
  (RNN): RNN(768, 256, num_layers=8, batch_first=True)
  (GRU): GRU(768, 256, num_layers=8, batch_first=True)
  (linear): Linear(in_features=256, out_features=68, bias=True)
  (softmax): Softmax(dim=2)
)>
0


ValueError: only one element tensors can be converted to Python scalars

In [45]:
def withoutattention():
    wandb.init(project='CS6910_DLAssignment3')
    config = wandb.config
    wandb.run.name = "withoutattention_cell_type_{}_bidirec_{}_layers_{}_batchsize_{}_hidden_{}".format(config.cell_type,config.bidirectional,config.no_of_layers,config.batchsize,config.hidden_size)
    hidden_size = config.hidden_size
    char_embed_size = config.input_embedding_size
    no_of_layers = config.no_of_layers
    epochs = config.epochs
    batchsize = config.batchsize
    dropout = config.dropout
    rnn = config.cell_type
    Encoder1,Decoder1 = train(batchsize,hidden_size,char_embed_size,no_of_layers,dropout,epochs,rnn)
    free_gpu_cache()

In [46]:
def main():
    wandb.init(project='CS6910_DLAssignment3')
    config = wandb.config
    wandb.run.name = "attention_cell_type_{}_bidirec_{}_layers_{}_batchsize_{}_hidden_{}".format(config.cell_type,config.bidirectional,config.no_of_layers,config.batchsize,config.hidden_size)
    hidden_size = config.hidden_size
    char_embed_size = config.input_embedding_size
    no_of_layers = config.no_of_layers
    epochs = config.epochs
    batchsize = config.batchsize
    dropout = config.dropout
    rnn = config.cell_type
    Encoder1,Decoder1 = attentiontrain(batchsize,hidden_size,char_embed_size,no_of_layers,dropout,epochs,rnn)
    free_gpu_cache()

In [47]:
!pip install GPUtil

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7409 sha256=0d748e0667fd19faa1842a901a249bcd83a15a435064959e91d1f68c25d7280e
  Stored in directory: /root/.cache/pip/wheels/b1/e7/99/2b32600270cf23194c9860f029d3d5db075f250bc39028c045
Successfully built GPUtil


In [57]:
free_gpu_cache()

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
|  1 |  0% |  0% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  4% |  1% |
|  1 |  0% |  0% |


In [50]:
sweep_configuration = {
    'method' : 'bayes',
    'metric' : { 'goal' : 'maximize',
    'name' : 'validation_accuracy'},
    'parameters':{
        'batchsize' : {'values' : [256]},
        'input_embedding_size' : {'values' : [512]},
        'no_of_layers' : {'values' : [2]},
        'hidden_size' : {'values' : [1024]},
        'cell_type' : {'values' : ['LSTM']},
        'bidirectional' : {'values' : ['Yes']},
        'dropout' : {'values' : [0.3]},
        'epochs' : {'values' : [30]}
    }
}
sweep_id = wandb.sweep(sweep = sweep_configuration,project = 'CS6910_DLAssignment3')
wandb.agent(sweep_id,function=main,count = 1)

Create sweep with ID: xf69w1mf
Sweep URL: https://wandb.ai/cs22m031/CS6910_DLAssignment3/sweeps/xf69w1mf


wandb: Agent Starting Run: woa4lgy1 with config:
wandb: 	batchsize: 256
wandb: 	bidirectional: Yes
wandb: 	cell_type: GRU
wandb: 	dropout: 0.3
wandb: 	epochs: 30
wandb: 	hidden_size: 1024
wandb: 	input_embedding_size: 512
wandb: 	no_of_layers: 2
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


<bound method Module.parameters of Encoder(
  (embedding): Embedding(29, 512)
  (drop): Dropout(p=0.3, inplace=False)
  (LSTM): LSTM(512, 1024, num_layers=2, batch_first=True, bidirectional=True)
  (RNN): RNN(512, 1024, num_layers=2, batch_first=True, bidirectional=True)
  (GRU): GRU(512, 1024, num_layers=2, batch_first=True, bidirectional=True)
)>
<bound method Module.parameters of Attention(
  (embedding): Embedding(68, 512)
  (drop): Dropout(p=0.3, inplace=False)
  (U): Linear(in_features=1024, out_features=1024, bias=False)
  (W): Linear(in_features=1024, out_features=1024, bias=False)
  (V): Linear(in_features=1024, out_features=1, bias=False)
  (LSTM): LSTM(1536, 1024, num_layers=2, batch_first=True)
  (RNN): RNN(1536, 1024, num_layers=2, batch_first=True)
  (GRU): GRU(1536, 1024, num_layers=2, batch_first=True)
  (linear): Linear(in_features=1024, out_features=68, bias=True)
  (softmax): Softmax(dim=2)
)>
0


/opt/conda/lib/python3.7/site-packages/torch/_tensor.py:760: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28


wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
sweep_configuration = {
    'method' : 'bayes',
    'metric' : { 'goal' : 'maximize',
    'name' : 'validation_accuracy'},
    'parameters':{
        'batchsize' : {'values' : [64,128,256,512]},
        'input_embedding_size' : {'values' : [128,256,512,1024]},
        'no_of_layers' : {'values' : [1,2,3,4]},
        'hidden_size' : {'values' : [128,256,512,1024]},
        'cell_type' : {'values' : ['LSTM']},
        'bidirectional' : {'values' : ['Yes']},
        'dropout' : {'values' : [0.3,0.4,0.5]},
        'epochs' : {'values' : [10,20,30]}
    }
}
sweep_id = wandb.sweep(sweep = sweep_configuration,project = 'CS6910_DLAssignment3')
wandb.agent(sweep_id,function=main,count = 20)

Create sweep with ID: 15q3jr0g
Sweep URL: https://wandb.ai/cs22m031/CS6910_DLAssignment3/sweeps/15q3jr0g


wandb: Agent Starting Run: cew7rq2i with config:
wandb: 	batchsize: 64
wandb: 	bidirectional: Yes
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	hidden_size: 1024
wandb: 	input_embedding_size: 256
wandb: 	no_of_layers: 4
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


<bound method Module.parameters of Encoder(
  (embedding): Embedding(29, 256)
  (drop): Dropout(p=0.3, inplace=False)
  (LSTM): LSTM(256, 1024, num_layers=4, batch_first=True, bidirectional=True)
  (RNN): RNN(256, 1024, num_layers=4, batch_first=True, bidirectional=True)
  (GRU): GRU(256, 1024, num_layers=4, batch_first=True, bidirectional=True)
)>
<bound method Module.parameters of Attention(
  (embedding): Embedding(68, 256)
  (drop): Dropout(p=0.3, inplace=False)
  (U): Linear(in_features=1024, out_features=1024, bias=False)
  (W): Linear(in_features=1024, out_features=1024, bias=False)
  (V): Linear(in_features=1024, out_features=1, bias=False)
  (LSTM): LSTM(1280, 1024, num_layers=4, batch_first=True)
  (RNN): RNN(1280, 1024, num_layers=4, batch_first=True)
  (GRU): GRU(1280, 1024, num_layers=4, batch_first=True)
  (linear): Linear(in_features=1024, out_features=68, bias=True)
  (softmax): Softmax(dim=2)
)>
0
1
2
